Pre-processing updates 
* Add scene detection
* Convert coordinates to textual grids -- Aaryan Kangte
* ImageNet PreTrained model rather than YOLO
* Color? -- Median color of object

* scene is beach {object is person object color red object location top right} {object is car object color black object location center}

In [18]:
# try:
#     import transformers
# except:
#     !pip install transformers -qU

# try:
#     import torchmetrics
# except:
#     !pip install torchmetrics -qU

# !pip install transformers -qU
# !pip install torchmetrics -qU
# !pip install tqdm

In [19]:
import ast
import warnings
import re
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
from tqdm.notebook import tqdm_notebook
from torchmetrics.functional import bleu_score
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import wandb
warnings.filterwarnings("ignore")

from transformers import BertTokenizer, BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# Log in to wandb and initialize a new run in the project 'image_captioning_project'
!wandb login 038b286a5767ec07105f734fc1d449b228421e9d
wandb.init(project='image_captioning_project', entity='image_caption', name='bert_embedding_gru_final')

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▅▃▂▂▂▂▁▁▁
val_loss,▃▁█▃▃▄▃▄▄▄
epoch,9
train_loss,1.72704
val_loss,8.72641


In [32]:
def load_cleaned_text_data():
    with open("../../data/x_bert.txt", 'r') as f:
        x_text = f.readlines()

    x_text_cleaned = []
    for x in x_text:
        x_text_cleaned.append(x.split('\n')[0])

    with open("../../data/y.txt", 'r') as f:
        y_file = f.readlines()

    y_list_temp = []
    for y_val in y_file:
        y_list_temp.append((y_val.split(" \n")[0]))

    y_list = []
    for y in y_list_temp:
        x = y
        x = ast.literal_eval(x)
        y_list.append(x)

    y_list_clean = []
    for y in y_list:
        for i in range(len(y)):
            y[i] = re.sub(r'[^\w\s]', '', y[i])
            y[i] = y[i].lower()
        y_list_clean.append(y)

    return x_text_cleaned, y_list_clean

x_text, y_text = load_cleaned_text_data()

In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [34]:
x_text_marked = [] # list of strings (marked sentences) = '[CLS] + sentence + [SEP]'
x_token_ids_list = [] # list of tensors
x_segment_ids_list = [] # list of tensors

for x in x_text:
    x_marked = "[CLS] " + x + " [SEP]"
    x_text_marked.append(x_marked)
    x_tokenized = tokenizer.tokenize(x_marked) # tokenize the sentence into words
    
    x_token_ids = tokenizer.convert_tokens_to_ids(x_tokenized) # convert the words to their index in the BERT Vocabulary
    x_segment_ids = [1] * len(x_token_ids) # there is only one sentence in the input

    x_token_ids = torch.tensor([x_token_ids]) # convert the list to a pytorch tensor
    x_segment_ids = torch.tensor([x_segment_ids]) # convert the list to a pytorch tensor

    x_token_ids_list.append(x_token_ids)
    x_segment_ids_list.append(x_segment_ids)


y_text_marked = [] # list of strings (marked sentences) = '[CLS] + sentence + [SEP]'
y_token_ids_list = [] # list of tensors
y_segment_ids_list = [] # list of tensors

for y in y_text:
    y_text_single_marked = []
    y_token_ids_single_list = []
    y_segment_ids_single_list = []
    
    for i in range(len(y)):
        y_marked = "[CLS] " + y[i] + " [SEP]"
        y_text_single_marked.append(y_marked)
        y_tokenized = tokenizer.tokenize(y_marked) # tokenize the sentence into words
        
        y_token_ids = tokenizer.convert_tokens_to_ids(y_tokenized) # convert the words to their index in the BERT Vocabulary
        y_segment_ids = [1] * len(y_token_ids) # there is only one sentence in the input

        y_token_ids = torch.tensor([y_token_ids]) # convert the list to a pytorch tensor
        y_segment_ids = torch.tensor([y_segment_ids]) # convert the list to a pytorch tensor

        y_token_ids_single_list.append(y_token_ids)
        y_segment_ids_single_list.append(y_segment_ids)

    y_text_marked.append(y_text_single_marked)
    y_token_ids_list.append(y_token_ids_single_list)
    y_segment_ids_list.append(y_segment_ids_single_list)

KeyboardInterrupt: 

In [ ]:
train_x_text_marked, val_x_text_marked, train_y_text_marked, val_y_text_marked = train_test_split(x_text_marked, y_text_marked,
                                                                                                  test_size=0.2,
                                                                                                  random_state=42)
test_x_text_marked, val_x_text_marked, test_y_text_marked, val_y_text_marked = train_test_split(train_x_text_marked, train_y_text_marked,
                                                                                                  test_size=0.25,
                                                                                                  random_state=42)
# --------------------------------------------------------------
train_x_token_ids_list, val_x_token_ids_list, train_y_token_ids_list, val_y_token_ids_list = train_test_split(x_token_ids_list, y_token_ids_list,
                                                                                                  test_size=0.2,
                                                                                                  random_state=42)
test_x_token_ids_list, val_x_token_ids_list, test_y_token_ids_list, val_y_token_ids_list = train_test_split(train_x_token_ids_list, train_y_token_ids_list,
                                                                                                  test_size=0.25,
                                                                                                  random_state=42)
# --------------------------------------------------------------
train_x_segment_ids_list, val_x_segment_ids_list, train_y_segment_ids_list, val_y_segment_ids_list = train_test_split(x_segment_ids_list, y_segment_ids_list,
                                                                                                  test_size=0.2,
                                                                                                  random_state=42)
test_x_segment_ids_list, val_x_segment_ids_list, test_y_segment_ids_list, val_y_segment_ids_list = train_test_split(train_x_segment_ids_list, train_y_segment_ids_list,
                                                                                                  test_size=0.25,
                                                                                                  random_state=42)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size=768):
        super(EncoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.embedding = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input_token_ids, input_segment_ids, hidden):
        embedded = self.get_bert_embedding(input_token_ids, input_segment_ids).view(1, -1, 768)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def get_bert_embedding(self, input_token_ids, input_segment_ids):
        bert_outputs = self.embedding(input_token_ids, input_segment_ids)

        bert_hidden_states = bert_outputs[2]
        bert_token_embeddings = torch.stack(bert_hidden_states, dim=0)
        bert_token_embeddings = torch.squeeze(bert_token_embeddings, dim=1)
        bert_token_embeddings = bert_token_embeddings.permute(1,0,2)

        # second to last layer embeddings
        bert_tokens_second_to_last  = []

        for token in bert_token_embeddings:
            bert_tokens_second_to_last.append(token[-2])

        bert_tokens_second_to_last = torch.stack(bert_tokens_second_to_last, dim=0)
        
        return bert_tokens_second_to_last
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size=768, output_size=tokenizer.vocab_size):
        super(DecoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_token_ids, input_segment_ids, hidden):
        output = self.get_bert_embedding(input_token_ids, input_segment_ids).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def get_bert_embedding(self, input_token_ids, input_segment_ids):
        bert_outputs = self.embedding(input_token_ids, input_segment_ids)

        bert_hidden_states = bert_outputs[2]
        bert_token_embeddings = torch.stack(bert_hidden_states, dim=0)
        bert_token_embeddings = torch.squeeze(bert_token_embeddings, dim=1)
        bert_token_embeddings = bert_token_embeddings.permute(1,0,2)

        # second to last layer embeddings
        bert_tokens_second_to_last  = []

        for token in bert_token_embeddings:
            bert_tokens_second_to_last.append(token[-2])

        bert_tokens_second_to_last = torch.stack(bert_tokens_second_to_last, dim=0)
        
        return bert_tokens_second_to_last

In [ ]:
# Create a Model Class that combines both encoder and decoder and the entire FORWARD function
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.encoder = EncoderRNN().to(device)
        clear_output()
        self.decoder = DecoderRNN().to(device)
        clear_output()

        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr=0.001)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr=0.001)

        self.criterion = nn.NLLLoss()

        for param in self.encoder.embedding.parameters():
            param.requires_grad = False

        for param in self.encoder.embedding.encoder.layer[11].parameters():
            param.requires_grad = True

        for param in self.decoder.embedding.parameters():
            param.requires_grad = False

        encoder_total_params = sum(p.numel() for p in self.encoder.parameters() if p.requires_grad)
        decoder_total_params = sum(p.numel() for p in self.decoder.parameters() if p.requires_grad)

        print(f'Encoder trainable parameters: {encoder_total_params:,}')
        print(f'Decoder trainable parameters: {decoder_total_params:,}')

        self.config = {
            'encoder_embeddings': 'bert-base-uncased',
            'decoder_embeddings': 'bert-base-uncased',
            'encoder_hidden_size': 768,
            'decoder_hidden_size': 768,
            'decoder_output_size': tokenizer.vocab_size,
            'encoder_learning_rate': 0.001,
            'decoder_learning_rate': 0.001,
            'encoder_optimizer': 'Adam',
            'decoder_optimizer': 'Adam',
            'criterion': 'NLLLoss',
            'batch_size': 1,
            'teacher_forcing_ratio': 0.5,
        }
        wandb.config.update(self.config)

    def forward(self, input_token_ids, input_segment_ids, y_token_ids_list):
        input_length = input_token_ids.size()[1]
        target_length = y_token_ids_list[0].size()[1]
        n_captions = len(y_token_ids_list)

        encoder_hidden = self.encoder.init_hidden()

        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        for ei in range(input_length):
            encoder_output, encoder_hidden = self.encoder(input_token_ids[0][ei].view(1, -1).to(device),
                                                          input_segment_ids[0][ei].view(1, -1).to(device),
                                                          encoder_hidden)

        decoder_hidden = encoder_hidden
        decoder_input_token_ids = torch.tensor(tokenizer.cls_token_id, device=device).view(1, -1)
        decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

        teacher_forcing = True if random.random() < 0.5 else False
        word_loss_list = []

# Gradient optimization on each example after accumulating gradients of 1 sentence
        for di in range(target_length):
            if teacher_forcing:
                decoder_output, decoder_hidden = self.decoder(decoder_input_token_ids,
                                                              decoder_input_segment_ids,
                                                              decoder_hidden)
                decoder_input_token_ids = y_token_ids_list[0][0][di].view(1, -1).to(device)
                decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

            else:
                decoder_output, decoder_hidden = self.decoder(decoder_input_token_ids.to(device),
                                                              decoder_input_segment_ids.to(device),
                                                              decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input_token_ids = torch.tensor(topi.item(), device=device).view(1, -1)
                decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

            loss = self.criterion(decoder_output, y_token_ids_list[0][0][di].view(1).to(device))
            word_loss_list.append(loss)

        ex_loss = torch.mean(torch.stack(word_loss_list))  # sentence_loss = ex_loss
        # train_ex_loss_list.append(ex_loss)

        ex_loss.backward()
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()

        return ex_loss

    def validate(self, input_token_ids, input_segment_ids, y_token_ids_list, MAX_LENGTH=50):
        input_length = input_token_ids.size()[1]
        target_length = y_token_ids_list[0].size()[1]
        n_captions = len(y_token_ids_list)

        with torch.no_grad():
            encoder_hidden = self.encoder.init_hidden()

            for ei in range(input_length):
                encoder_output, encoder_hidden = self.encoder(input_token_ids[0][ei].view(1, -1).to(device),
                                                              input_segment_ids[0][ei].view(1, -1).to(device),
                                                              encoder_hidden)

            decoder_hidden = encoder_hidden
            decoder_input_token_ids = torch.tensor(tokenizer.cls_token_id, device=device).view(1, -1)
            decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

            teacher_forcing = False
            word_loss_list = []
            predicted_token_ids = []

            for di in range(MAX_LENGTH-1):
                if teacher_forcing:
                    decoder_output, decoder_hidden = self.decoder(decoder_input_token_ids,
                                                                  decoder_input_segment_ids,
                                                                  decoder_hidden)
                    decoder_input_token_ids = y_token_ids_list[0][0][di].view(1, -1).to(device)
                    decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

                else:
                    decoder_output, decoder_hidden = self.decoder(decoder_input_token_ids.to(device),
                                                                  decoder_input_segment_ids.to(device),
                                                                  decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input_token_ids = torch.tensor(topi.item(), device=device).view(1, -1)
                    decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)
                    predicted_token_ids.append(topi.item())
                
                loss = self.criterion(decoder_output, y_token_ids_list[0][0][di].view(1).to(device) if di < target_length else torch.tensor([tokenizer.pad_token_id], device=device))
                word_loss_list.append(loss)
                predicted_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(predicted_token_ids))

                if predicted_token_ids[-1] == tokenizer.sep_token_id:
                    break
                    
                if di == MAX_LENGTH-2:
                    predicted_token_ids.append(tokenizer.sep_token_id)
                    break
                
            ex_loss = torch.mean(torch.stack(word_loss_list))

        return ex_loss, predicted_sentence

    def predict(self, input_token_ids, input_segment_ids, MAX_LENGTH=50):
        input_length = input_token_ids.size()[1]

        with torch.no_grad():
            encoder_hidden = self.encoder.init_hidden()

            for ei in range(input_length):
                encoder_output, encoder_hidden = self.encoder(input_token_ids[0][ei].view(1, -1).to(device),
                                                              input_segment_ids[0][ei].view(1, -1).to(device),
                                                              encoder_hidden)

            decoder_hidden = encoder_hidden
            decoder_input_token_ids = torch.tensor(tokenizer.cls_token_id, device=device).view(1, -1)
            decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

            teacher_forcing = False
            predicted_token_ids = []

            for di in range(MAX_LENGTH-1):
                if teacher_forcing:
                    decoder_output, decoder_hidden = self.decoder(decoder_input_token_ids,
                                                                  decoder_input_segment_ids,
                                                                  decoder_hidden)
                    decoder_input_token_ids = y_token_ids_list[0][0][di].view(1, -1).to(device)
                    decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

                else:
                    decoder_output, decoder_hidden = self.decoder(decoder_input_token_ids.to(device),
                                                                  decoder_input_segment_ids.to(device),
                                                                  decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input_token_ids = torch.tensor(topi.item(), device=device).view(1, -1)
                    decoder_input_segment_ids = torch.tensor([[1] * len(decoder_input_token_ids)], device=device)

                predicted_token_ids.append(decoder_input_token_ids)
                
                if predicted_token_ids[-1] == tokenizer.sep_token_id:
                    break
                    
                if di == MAX_LENGTH-2:
                    predicted_token_ids.append(tokenizer.sep_token_id)
                    break

        input_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_token_ids[0]))
        predicted_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(predicted_token_ids))

        return input_sentence, predicted_sentence

    def test(self, test_loader, MAX_LENGTH=50):
        '''
        Args:
        test_loader: 3-tuple of (x_token_ids, x_segment_ids, y_token_ids_list)
        Returns: 
        4-tuple (test_loss, input_sentence, predicted_sentence, target_sentence)
        '''
        self.encoder.eval()
        self.decoder.eval()
        test_loss = 0

        input_sentences_list = []
        predicted_sentences_list = []
        target_sentences_list = []

        test_input_token_ids_list, test_input_segment_ids_list, test_y_token_ids_list_list = test_loader

        for input_token_ids, input_segment_ids, y_token_ids_list in zip(test_input_token_ids_list, test_input_segment_ids_list, test_y_token_ids_list_list):
            ex_loss, predicted_sentence = self.validate(input_token_ids, input_segment_ids, y_token_ids_list, MAX_LENGTH)
            test_loss += ex_loss.item()

            input_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_token_ids[0]))
            target_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_token_ids_list[0][0]))

            input_sentences_list.append(input_sentence)
            predicted_sentences_list.append(predicted_sentence)
            target_sentences_list.append(target_sentence)

        return test_loss / len(test_loader), input_sentences_list, predicted_sentences_list, target_sentences_list

    def save(self, path, epoch, loss):
        torch.save({
            'epoch': epoch,
            'encoder_state_dict': self.encoder.state_dict(),
            'decoder_state_dict': self.decoder.state_dict(),
            'encoder_state_dict': self.encoder_optimizer.state_dict(),
            'decoder_state_dict': self.decoder_optimizer.state_dict(),
            'loss': loss,
            'name': 'BERT_2nd-Last_GRU'
        }, f'{path}.pth')

    def copy_to_drive(self, path):
        !cp ${path}.pth "/content/drive/Models/${path}.pth"

    def log_wandb(self, train_epoch_loss, val_epoch_loss, epoch):
        wandb.log({
            'train_loss': train_epoch_loss,
            'val_loss': val_epoch_loss,
            'epoch': epoch
        })


In [ ]:
model = Model().to(device)

In [ ]:
# x_token_ids = train_x_token_ids_list[0][0].view(1, -1).to(device) # 1 word token id
# x_segment_ids = train_x_segment_ids_list[0][0].view(1, -1).to(device) # 1 word segment idx
# y_token_ids_list = train_y_token_ids_list[0] # 1 word token id

# # loss = model(x_token_ids, x_segment_ids, y_token_ids_list)
# input_sentence, predicted_sentence = model.predict(x_token_ids, x_segment_ids)
# print(input_sentence)
# print(predicted_sentence)

In [ ]:
EPOCHS = 25
N_TRAIN = 4000
N_VAL = 500
PATH = 'model'
BEST_PATH = 'best_model'
PATIENCE = 10
early_stopper = 0
use_early_stopping = False

train_epoch_loss_list = []
val_epoch_loss_list = []

min_val_loss = 10000

for epoch in range(EPOCHS):
    train_ex_loss_list = []
    val_ex_loss_list = []

    # TRAIN STEP:
    for i in tqdm_notebook(range(N_TRAIN)):
        x_token_ids = train_x_token_ids_list[i].to(device)
        x_segment_ids = train_x_segment_ids_list[i].to(device)

        y_token_ids_list = train_y_token_ids_list[i]
        y_segment_ids_list = train_y_segment_ids_list[i]

        ex_loss = model(x_token_ids, x_segment_ids, y_token_ids_list)
        train_ex_loss_list.append(ex_loss)

    train_epoch_loss = torch.mean(torch.stack(train_ex_loss_list))
    train_epoch_loss_list.append(train_epoch_loss.item())

    # VAL STEP:
    for i in tqdm_notebook(range(N_VAL)):
        x_token_ids = val_x_token_ids_list[i].to(device)
        x_segment_ids = val_x_segment_ids_list[i].to(device)

        y_token_ids_list = val_y_token_ids_list[i]
        y_segment_ids_list = val_y_segment_ids_list[i]

        input_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(x_token_ids[0]))
        target_sentence = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_token_ids_list[0][0]))

        ex_loss, predicted_sentence = model.validate(x_token_ids, x_segment_ids, y_token_ids_list, MAX_LENGTH=50)
        val_ex_loss_list.append(ex_loss)

    val_epoch_loss = torch.mean(torch.stack(val_ex_loss_list))
    val_epoch_loss_list.append(val_epoch_loss.item())

    print(f'EPOCH: {epoch+1} \t train_loss: {train_epoch_loss_list[-1]:.3f} \t val_loss: {val_epoch_loss_list[-1]:.3f}')
    print(f'input: {input_sentence}')
    print(f'target: {target_sentence}')
    print(f'predicted: {predicted_sentence}')
    print()

    model.log_wandb(train_epoch_loss, val_epoch_loss, epoch)

    model.save(PATH, epoch, train_epoch_loss)
    # model.copy_to_drive(PATH)

    if val_epoch_loss < min_val_loss:
        min_val_loss = val_epoch_loss
        min_val_loss_epoch = epoch
        model.save(BEST_PATH, epoch, train_epoch_loss)
        # model.copy_to_drive(BEST_PATH)

        
    if use_early_stopping:
        if epoch > 1:
            if val_epoch_loss > val_epoch_loss_list[-2]:
                early_stopper += 1

                if early_stopper == PATIENCE:
                    print('Early Stopping!')
                    print(f'Best Val Loss: {min_val_loss} at epoch {min_val_loss_epoch+1}')
                    break
            else:
                early_stopper = 0

In [ ]:
wandb.finish()

In [ ]:
test_loader = (test_x_token_ids_list, test_x_segment_ids_list, test_y_token_ids_list)

test_loss, test_input_sentences, test_predicted_sentences, test_target_sentences = model.test(test_loader)

In [ ]:
for i, (input_sentence, predicted_sentence, target_sentence) in enumerate(zip(test_input_sentences, test_predicted_sentences, test_target_sentences)):
    print(f'input: {input_sentence}')
    print(f'target: {target_sentence}')
    print(f'predicted: {predicted_sentence}')
    print()
    
    if i == 10:
        break